# Amp generator

Classes

In [1]:
import random
from Bio.SeqUtils.ProtParam import ProteinAnalysis
import itertools
import more_itertools
import pprint
import numpy as np
import pandas as pd
import math
import subprocess
import os
from tqdm.notebook import tqdm
import timeit
import multiprocessing as mp
from IPython.display import clear_output

In [17]:
#Raw sequence
class Raw_sequence:
    length_lower = 14
    length_upper = 14
    num_cat = 3
    gravy_lower = 0.1
    gravy_upper = 2.0
    cat = ['K', 'R']
    polar = ['S','T']
    hydrophobic = ['A','L', 'I']
    special = []
    raw_seq = []
    
    def get_length(self):
        if self.length_lower == self.length_upper:
            return self.length_lower
        else:
            return random.randrange(self.length_lower,self.length_upper)
    
    def add_cat(self):
        for i in range(0,self.num_cat):
            self.raw_seq.append(random.choice(self.cat))
        return self.raw_seq
    
    def add_other(self):
        other = self.polar + self.hydrophobic + self.special
        for i in range(self.num_cat, self.get_length()):
            self.raw_seq.append(random.choice(other))
        return self.raw_seq
    
    def make_raw_seq(self):
        self.add_cat()
        self.add_other()
        return self.raw_seq
    
    def calc_gravy(self):
        seq = ''.join(self.raw_seq)
        return str(ProteinAnalysis(seq).gravy())
    
    def check_gravy(self):
        if float(self.calc_gravy()) < self.gravy_lower or float(self.calc_gravy()) > self.gravy_upper:
            self.raw_seq.clear()
        else:
            self.raw_seq = self.raw_seq
        return self.raw_seq
    
    def gen_raw_seq(self):
        self.raw_seq.clear()
        self.make_raw_seq()
        self.check_gravy()
        while len(self.raw_seq) == 0:
            self.gen_raw_seq()
        else:
            return self.raw_seq
        
    def clear(self):
        self.raw_seq.clear()
        
    

In [136]:
#Ordered sequence
class Ordered_sequence:
    
    
    #raw_seq = list(Raw_sequence().gen_raw_seq())
    
    cat_resid_distance_from_N = 0 - 1
    cat_resid_distance_from_C = 1 - 1
    
    iterations = 1000
    
    #Need to inherit from other class
    cat = ['K', 'R']
    polar = ['S','T']
    hydrophobic = ['A','L', 'I']
    
    angle_cat_upper = 170
    angle_cat_lower = 10
    angle_polar_upper = 170
    angle_polar_lower = 10
    angle_hydrophobic_upper = 360
    angle_hydrophobic_lower = 180
    
    accepted_seq_list = []
    
    def set_raw_seq(self, seq):
        self.raw_seq = seq
    
    
    def gen_random_order(self):
        ordered_seq_set = set()
        for i in range(0,self.iterations):
            ordered_seq_set.add(more_itertools.random_permutation(self.raw_seq))
        return ordered_seq_set
    
    def check_terminal_distance(self):
        ordered_seq_list = list(self.gen_random_order())
        removal_seq_list = []
        
        for seq in ordered_seq_list:
            try:
                if seq.index('K') <= self.cat_resid_distance_from_N:
                        removal_seq_list.append(seq)
                elif seq[::-1].index('K') <= self.cat_resid_distance_from_C:
                        removal_seq_list.append(seq)
            except ValueError:
                pass
           
        
        for seq in ordered_seq_list:
            try:
                if seq.index('R') <= self.cat_resid_distance_from_N:
                    removal_seq_list.append(seq)
                elif seq[::-1].index('R') <= self.cat_resid_distance_from_C:
                    removal_seq_list.append(seq)
            except ValueError:
                pass

                    
        return (set(ordered_seq_list) - set(removal_seq_list))
                    
                    
    def calc_theoretical_wheel(self, seq):
        residue_orientation = np.arange(float(len(seq))) * -100
        residue_orientation = pd.DataFrame(residue_orientation, columns  = ['raw'])
          
        correction = []
  
        for i in range(0, len(seq)):
            correction.append(residue_orientation.iat[i,0] // 360)
  
        residue_orientation['correction factor'] = correction
  
        residue_orientation['x'] = residue_orientation['correction factor'] * 360
  
        residue_orientation['360 degree'] = residue_orientation['raw'] - residue_orientation['x']

        return pd.DataFrame(data = residue_orientation['360 degree'])
  
    
    
    def calc_hydrophobic_moment(self, seq):
        
        degrees = self.calc_theoretical_wheel(seq)
        degrees = np.deg2rad(degrees)
          
        sequence = pd.DataFrame(data = seq)
  
  
        scale = {'A':  0.25, 'R': -1.80, 'N': -0.64,
                'D': -0.72, 'C':  0.04, 'Q': -0.69,
                'E': -0.62, 'G':  0.16, 'H': -0.40,
                'I':  0.73, 'L':  0.53, 'K': -1.10,
                'M':  0.26, 'F':  0.61, 'P': -0.07,
                'S': -0.26, 'T': -0.18, 'W':  0.37,
                'Y':  0.02, 'V':  0.54}
  
        hm = []
  
        for i in range(0, len(sequence)):
            eisenberg = scale.get(sequence.iat[i,0])
            hm.append((eisenberg * math.cos(degrees.iat[i,0]), eisenberg * math.sin(degrees.iat[i,0])))
    
        hm = pd.DataFrame(data=hm, columns = ['x-vector', 'y-vector'])
  
        hm_vector = hm.sum()
  
        hm_vector = hm_vector.tolist()
  
        unit_vector_1 = hm_vector  / np.linalg.norm(hm_vector)
        unit_vector_2 = [1,0] / np.linalg.norm([1,0]) 
        unit_vector_3 = [0,1] / np.linalg.norm([0,1])
        unit_vector_4 = [0,-1] / np.linalg.norm([0,-1])
  
        dot_product_1 = np.dot(unit_vector_1, unit_vector_2)
        dot_product_2 = np.dot(unit_vector_1, unit_vector_3)
        dot_product_3 = np.dot(unit_vector_1, unit_vector_4)
  
        angle1 = np.arccos(dot_product_1)
        angle1 = np.rad2deg(angle1)
        angle2 = np.arccos(dot_product_2)
        angle2 = np.rad2deg(angle2)
        angle3 = np.arccos(dot_product_3)
        angle3 = np.rad2deg(angle3)  
  
        if angle2 > angle3:
            corrected_angle = 360 - angle1
        else:
            corrected_angle = angle1  
    
        return corrected_angle
    
    
    
    def correct_orientations(self, seq):
    
        resiude_angles = self.calc_theoretical_wheel(seq)
        hm_angle = self.calc_hydrophobic_moment(seq)
        
        sequence = pd.DataFrame(data = seq, columns = ['Residues'])
  
        correction = 270 - hm_angle
  
        corrected_angles = resiude_angles + correction
  
        final_angles = []
  
        for i in range(0,len(corrected_angles)):
            if corrected_angles.iat[i,0] > 360:
                final_angles.append((corrected_angles.iat[i,0] - 360))
            elif corrected_angles.iat[i,0] < 0:
                final_angles.append((corrected_angles.iat[i,0] + 360))
            else:
                final_angles.append(corrected_angles.iat[i,0])
  
        final_angles = pd.DataFrame(data = final_angles, columns = ['360 angle'])
    
        results = pd.concat([sequence, final_angles], axis=1)
  
        return results

        
    
    
    def get_residue_angles(self):
        #print("Calculating helical wheels")
        seq_list = list(self.check_terminal_distance())
        x = []
        for seq in tqdm(seq_list):
            x.append(self.correct_orientations(seq))
        return x

    
    def check_angles(self):
        seq_list = self.get_residue_angles()
        #print("Checking against orientation criteria")
        save_list = []
        for i in range(0,len(seq_list)):
            sequence = seq_list[i]
            k=0
            for x in range(0,len(sequence)):
                if sequence.iat[x,0] in self.cat and self.angle_cat_lower < sequence.iat[x,1] < self.angle_cat_upper or sequence.iat[x,0] in self.polar and self.angle_polar_lower < sequence.iat[x,1] < self.angle_polar_upper or sequence.iat[x,0] in self.hydrophobic and self.angle_hydrophobic_lower < sequence.iat[x,1] < self.angle_hydrophobic_upper:
                    k=k+1
            if k == len(sequence):
                save_list.append(sequence)
                    
        #print("%s sequences out of %s permutations found based on the sequence %s that are in agreement with the orientational and positional criteria" % (len(save_list),self.iterations, seq_list[0]['Residues'].tolist()))        
        
        
        return save_list


In [4]:
import matplotlib.lines as lines
import matplotlib.patches as patches
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from scipy.stats.kde import gaussian_kde

from modlamp.core import count_aas, load_scale
from modlamp.descriptors import PeptideDescriptor, GlobalDescriptor


def helical_wheel():

    # color mappings
    aa = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']
    f_rainbow = ['#3e3e28', '#ffcc33', '#b30047', '#b30047', '#ffcc33', '#3e3e28', '#80d4ff', '#ffcc33', '#0047b3',
                 '#ffcc33', '#ffcc33', '#b366ff', '#29a329', '#b366ff', '#0047b3', '#ff66cc', '#ff66cc', '#ffcc33',
                 '#ffcc33', '#ffcc33']

    t_rainbow = ['w', 'k', 'w', 'w', 'k', 'w', 'k', 'k', 'w', 'k', 'k', 'k', 'k', 'k', 'w', 'k', 'k', 'k', 'k', 'k']
    
    d_eisberg = {'A':  [0.25], 'R': [-1.80], 'N': [-0.64],
                'D': [-0.72], 'C':  [0.04], 'Q': [-0.69],
                'E': [-0.62], 'G':  [0.16], 'H': [-0.40],
                'I':  [0.73], 'L':  [0.53], 'K': [-1.10],
                'M':  [0.26], 'F':  [0.61], 'P': [-0.07],
                'S': [-0.26], 'T': [-0.18], 'W':  [0.37],
                'Y':  [0.02], 'V':  [0.54]}
    
    data = Ordered_sequence().check_angles()
    pprint.pprint(data[0])
    
    deg = []
    sequence = []
    
    for i in range(0,len(data[0])):
        deg.append(data[0].iat[i,1])
        sequence.append(data[0].iat[i,0]) 
    
    lw = [2.] * (len(sequence) - 1)
    
    df = dict(zip(aa, f_rainbow))
    dt = dict(zip(aa, t_rainbow))

    rad = np.radians(deg)
    
    # dict for coordinates and eisenberg values
    d_hydro = dict(zip(rad, [0.] * len(rad)))
    
    # create figure
    fig = plt.figure(frameon=False, figsize=(10, 10))
    ax = fig.add_subplot(111)
    old = None
    hm = list()
    
    # iterate over sequence
    for i, r in enumerate(rad):
        new = (np.cos(r), np.sin(r))  # new AA coordinates
        if old is not None:
            line = lines.Line2D((old[0], new[0]), (old[1], new[1]), transform=ax.transData, color='k',
                                    linewidth=lw[i - 1])
            line.set_zorder(1)  # 1 = level behind circles
            ax.add_line(line)

        
        # plot circles
        circ = patches.Circle(new, radius=0.1, transform=ax.transData, edgecolor='k', facecolor=df[sequence[i]])
        circ.set_zorder(2)  # level in front of lines
        ax.add_patch(circ)
        
        # check if N- or C-terminus and add subscript, then plot AA letter
        if i == 0:
            ax.text(new[0], new[1], sequence[i] + '$_N$', va='center', ha='center', transform=ax.transData,
                    size=32, color=dt[sequence[i]], fontweight='bold')
        elif i == len(sequence) - 1:
            ax.text(new[0], new[1], sequence[i] + '$_C$', va='center', ha='center', transform=ax.transData,
                    size=32, color=dt[sequence[i]], fontweight='bold')
        else:
            ax.text(new[0], new[1], sequence[i], va='center', ha='center', transform=ax.transData,
                    size=36, color=dt[sequence[i]], fontweight='bold')
        print(type(sequence))
        eb = d_eisberg[sequence[i]][0]  # eisenberg value for this AA
        hm.append([eb * new[0], eb * new[1]])  # save eisenberg hydrophobicity vector value to later calculate HM
        
        
        old = (np.cos(r), np.sin(r))  # save as previous coordinates

    # draw hydrophobic moment arrow if moment option
    v_hm = np.sum(np.array(hm), 0)
    x = .0333 * v_hm[0]
    y = .0333 * v_hm[1]
    ax.arrow(0., 0., x, y, head_width=0.04, head_length=0.03, transform=ax.transData,
            color='k', linewidth=6.)
    desc = PeptideDescriptor(sequence)  # calculate hydrophobic moment
    desc.calculate_moment()
    if abs(x) < 0.2 and y > 0.:  # right positioning of HM text so arrow does not cover it
        z = -0.2
    else:
        z = 0.2
    plt.text(0., z, str(round(desc.descriptor[0][0], 3)), fontdict={'fontsize': 20, 'fontweight': 'bold',
                                                                        'ha': 'center'})
    unit_vector_1 = v_hm  / np.linalg.norm(v_hm)
    unit_vector_2 = [1,0] / np.linalg.norm([1,0]) 
    dot_product = np.dot(unit_vector_1, unit_vector_2)
    angle = np.arccos(dot_product)
    angle = np.rad2deg(angle)
  

    # plot shape
    ax.set_xlim(-1.2, 1.2)
    ax.set_ylim(-1.2, 1.2)
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    cur_axes = plt.gca()
    cur_axes.axes.get_xaxis().set_visible(False)
    cur_axes.axes.get_yaxis().set_visible(False)
    plt.tight_layout()
    
    

    plt.show() 

In [156]:
import multiprocessing as mp


def gen_seq(num):
    seq_list_raw = []
    for i in range(num):
        seq_list_raw.append(list(Raw_sequence().gen_raw_seq()))
    return seq_list_raw
    
    
def order_seq(seq_list_raw):
    seq_list_ordered = []
    for i in tqdm(seq_list_raw):
        x = Ordered_sequence()
        x.set_raw_seq(i)
        seq_list_ordered.append(list(x.check_angles()))
    
    seq_list_ordered = list(filter(None, seq_list_ordered))
    return seq_list_ordered


def order_seq_mp(seq):
    x = Ordered_sequence()
    x.set_raw_seq(seq)
    return (list(x.check_angles()))


def main(num):
    seq_list_raw = gen_seq(num)
    pool = mp.Pool(mp.cpu_count())
    results = tqdm(pool.imap(order_seq_mp,seq_list_raw), total=len(seq_list_raw))
    pool.close()
    results = list(filter(None, results))
    
    return results

        

In [157]:

pprint.pprint(main(100))

  0%|          | 0/100 [00:00<?, ?it/s]

[[   Residues   360 angle
0         L  328.086513
1         L  228.086513
2         R  128.086513
3         K   28.086513
4         A  288.086513
5         L  188.086513
6         S   88.086513
7         I  348.086513
8         L  248.086513
9         T  148.086513
10        K   48.086513
11        I  308.086513
12        A  208.086513
13        T  108.086513],
 [   Residues   360 angle
0         S  131.049125
1         T   31.049125
2         A  291.049125
3         A  191.049125
4         T   91.049125
5         L  351.049125
6         I  251.049125
7         K  151.049125
8         R   51.049125
9         A  311.049125
10        I  211.049125
11        R  111.049125
12        T   11.049125
13        L  271.049125],
 [   Residues   360 angle
0         T   79.633002
1         I  339.633002
2         I  239.633002
3         S  139.633002
4         R   39.633002
5         L  299.633002
6         L  199.633002
7         R   99.633002
8         A  359.633002
9         I  259.633002
10    

Class inheritance


In [57]:
d1=Raw_sequence()
e1=Raw_sequence()

# print(d1.gen_raw_seq())
# print(e1.gen_raw_seq())

d2=Ordered_sequence()
s = ['A','A','A','K','K','K','K','A','A','A']
d2.set_raw_seq(s)
# e2=Ordered_sequence()

# print()

# print(d2.raw_seq)
# print(e2.raw_seq)

#pprint.pprint(d2.gen_random_order())

#test(10)



# print("gen_random_order")
# %timeit d2.gen_random_order()

# print("check_terminal_distance")
# %timeit d2.check_terminal_distance()

# seq = list("AKAKAKAKAKAKAKA")

# print("calc_theoretical_wheel")
# %timeit d2.calc_theoretical_wheel(seq)

# print("calc_hydrophobic_moment")
# %timeit d2.calc_hydrophobic_moment(seq)

# print("correct_orientations")
# %timeit d2.correct_orientations(seq)

# print("get_residue_angles")
# %timeit d2.get_residue_angles()

#print("check_angles")
d2.check_angles()

#run(2)

#seq= list(d1.gen_raw_seq())

#seq =['A','B','B','A']

#seq = ''.join(seq)
#seq = set(seq)

#print(seq)
#print(len(seq))
#print(len(set(itertools.permutations(seq,4))))
#print(set(itertools.permutations(seq,4)))

#print(d2.raw_seq,list(d2.gen_random_order()))
      #len(d2.check_terminal_distance()))
    
#pprint.pprint(len(d2.check_terminal_distance()))

#pprint.pprint(Ordered_sequence().get_residue_angles())


#pprint.pprint(d2.check_angles())

#run(2)

#test(10)
    
    
#x = list(d2.gen_random_order())
#print(x)

#for i in x:
   # print(i.index('K'))


Calculating helical wheels


  0%|          | 0/125 [00:00<?, ?it/s]

Checking against orientation criteria


  0%|          | 0/125 [00:00<?, ?it/s]

1 sequences out of 1000 permutations found based on the sequence ['K', 'K', 'A', 'K', 'A', 'A', 'A', 'K', 'A', 'A'] that are in agreement with the orientational and positional criteria


[  Residues   360 angle
 0        K   77.330574
 1        A  337.330574
 2        A  237.330574
 3        K  137.330574
 4        K   37.330574
 5        A  297.330574
 6        A  197.330574
 7        K   97.330574
 8        A  357.330574
 9        A  257.330574]

In [7]:
helical_wheel()

Calculating helical wheels


  0%|          | 0/755 [00:00<?, ?it/s]

Checking against orientation criteria


  0%|          | 0/755 [00:00<?, ?it/s]

1 sequences out of 1000 permutations found based on the sequence ['K', 'R', 'R', 'I', 'S', 'L', 'S', 'L', 'T', 'A', 'L', 'L', 'L', 'L'] that are in agreement with the orientational and positional criteria
[   Residues   360 angle
0         T   63.384301
1         I  323.384301
2         L  223.384301
3         R  123.384301
4         S   23.384301
5         L  283.384301
6         L  183.384301
7         R   83.384301
8         L  343.384301
9         L  243.384301
10        S  143.384301
11        K   43.384301
12        L  303.384301
13        A  203.384301]


AttributeError: 'list' object has no attribute 'iat'

In [ ]:
seq = ''.join(['K', 'K', 'R', 'S', 'L', 'T', 'L', 'A', 'T', 'A', 'A', 'L', 'L', 'L'])
print(seq)
print(str(ProteinAnalysis(seq).gravy()))